<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#第2章-k-近邻算法" data-toc-modified-id="第2章-k-近邻算法-1">第2章 k-近邻算法</a></span><ul class="toc-item"><li><span><a href="#本章内容" data-toc-modified-id="本章内容-1.1">本章内容</a></span></li><li><span><a href="#KNN概述" data-toc-modified-id="KNN概述-1.2">KNN概述</a></span></li><li><span><a href="#kNN原理" data-toc-modified-id="kNN原理-1.3">kNN原理</a></span></li><li><span><a href="#kNN流程" data-toc-modified-id="kNN流程-1.4">kNN流程</a></span></li><li><span><a href="#代码实现与解读" data-toc-modified-id="代码实现与解读-1.5">代码实现与解读</a></span></li><li><span><a href="#实施k-近邻算法" data-toc-modified-id="实施k-近邻算法-1.6">实施k-近邻算法</a></span><ul class="toc-item"><li><span><a href="#示例：使用k-近邻算法改进约会网站的配对系统" data-toc-modified-id="示例：使用k-近邻算法改进约会网站的配对系统-1.6.1">示例：使用k-近邻算法改进约会网站的配对系统</a></span><ul class="toc-item"><li><span><a href="#1.将文本记录转换为-NumPy-的解析程序" data-toc-modified-id="1.将文本记录转换为-NumPy-的解析程序-1.6.1.1">1.将文本记录转换为 NumPy 的解析程序</a></span></li><li><span><a href="#2.分析数据：使用Matplotlib画二维散点图" data-toc-modified-id="2.分析数据：使用Matplotlib画二维散点图-1.6.1.2">2.分析数据：使用Matplotlib画二维散点图</a></span></li><li><span><a href="#3.归一化数值" data-toc-modified-id="3.归一化数值-1.6.1.3">3.归一化数值</a></span></li><li><span><a href="#4.分类器的测试代码" data-toc-modified-id="4.分类器的测试代码-1.6.1.4">4.分类器的测试代码</a></span></li></ul></li></ul></li><li><span><a href="#KNN场景" data-toc-modified-id="KNN场景-1.7">KNN场景</a></span><ul class="toc-item"><li><span><a href="#举个例子" data-toc-modified-id="举个例子-1.7.1">举个例子</a></span></li></ul></li></ul></li></ul></div>

**标准简化内容，避繁就简，简化函数名称**

# 第2章 k-近邻算法  


## 本章内容

> * k-近邻分类算法
* 从文本文件中解析和导入数据
* 使用matplotlib创建扩散图
* 使用归一化数值



## KNN概述  

k-近邻（kNN，k-NearestNeighbor）算法主要是用来进行分类的  
* kNN算法特点  

>优点：精度高、对异常值不敏感、无数据输入假定  
缺点：计算复杂度高、空间复杂度高   
适用数据范围：数值型和标称型

## kNN原理


k-近邻（k-NearestNeighbour，简称$kNN$），常用于有监督学习。

kNN的工作原理：存在一个训练样本集合A，在给定测试样本B时，基于某种距离度量，找出训练集中A与测试样本B最靠近的k个训练样本（通常  $k\leq{20}$且为整数），之后，基于则$k$个训练样本的信息来预测种类或值。  


## kNN流程

* kNN开发流程    

>收集数据：任何方法    
准备数据：距离计算所需要的数值，最好是结构化的数据格式   
分析数据：任何方法    
训练算法：此步骤不适用于 k-近邻算法    
测试算法：计算错误率    
使用算法：输入样本数据和结构化的输出结果，然后运行 k-近邻算法判断输入数据分类属于哪个分类，最后对计算出的分类执行后续处理    



## 代码实现与解读

现在机器学习的很多算法都能调用scikit-learn中的包来实现，可能只需要一行命令。但最好实现一遍算法，加深理解。

在构造完整k-近邻算法之前，我们首先编写一些基本通用算法，编写以下代码

In [7]:
from numpy import *
import operator

def createData():
    group=array([[1.0,1.1],[1.0,1.0],[0,0],[0,0.1]])
    labels=['A','A','B','B']
    return group,labels   
group,labels=createData()
group,labels   

(array([[1. , 1.1],
        [1. , 1. ],
        [0. , 0. ],
        [0. , 0.1]]), ['A', 'A', 'B', 'B'])

## 实施k-近邻算法
** 伪代码如下：**

对测试样本点进行以下操作：  
>（1）计算已知类别数据集中的点与当前点之间的距离；   
（2）按照距离递增次序排序；   
（3）选取与当前距离最小的k个点；   
（4）确定前k个点所在类别的出现的频率；   
（5）返回前k个点出现频率最高的类别作为当前点的预测分类。   




In [3]:
from numpy import *
import operator 

#这是k-近邻算法主体分类代码
#四个参数的意义：
#inX：用来分类新的测试集
#dataSet：用来训练数据集的属性
#labels：用来训练数据集的标签
#k：k-近邻的k

def classify0(inX, dataSet, labels, k):
    dataSetSize = dataSet.shape[0]      #shape[0],返回dataSet的行数
    diffMat = tile(inX, (dataSetSize,1)) - dataSet
                                        #tile()函数实现将矩阵重复；在列方向上重复inX共1次
                                        #行向量上重复inX共dataSetSize次
                                        #实现对应数值相减
    sqDiffMat = diffMat**2              #平方
    
    sqDistances = sqDiffMat.sum(axis=1) #sum（）为求和，参数axis=1为行方向求和，参数axis=0为列方向求和
   
    distances = sqDistances**0.5        #开方

    sortedDistIndicies = distances.argsort()    #distances.atgsort()，将distance中的元素即距离，从小到大排序，并返回索引值
                                                #注意：sortedDistIndicies中存放的是索引值
    
    classCount={}                              #创建一个字典         
    for i in range(k):

        voteIlabel = labels[sortedDistIndicies[i]]          #返回距离排名第i的标签
        
        classCount[voteIlabel] = classCount.get(voteIlabel,0)+1         #dict.get(key,default=None)，返回字典中指定键key的值，
                                                                        #如果键key不在字典中，则返回默认值default的值
                                                                        #计算了类别的数目
       
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    
                                                                    #将classCount中的值进行排序
                                                                    #sorted(iterabel,cmp=None,key=None,reverser=False)
                                                            #cmp用于比较函数；key=operator.itergetter()根据字典的值进行排序
                                                            #key=operator=itemgetter(0)根据字典的键进行排序
                                                            #reverser=True降序或者reverse=False升序
  
    return sortedClassCount[0][0]    #返回要分类的类别

备注1：shape函数是numpy.core.fromnumeric中的函数，它的功能是读取矩阵的长度，比如shape[0]就是读取矩阵第一维度的长度。它的输入参数可以使一个整数表示维度，也可以是一个矩阵。

In [10]:
classify0([0,0],group,labels,3)

'B'

其中，在分类问题中，KNN用来预测种类。一般使用“投票法”，选择这k个样本中出现最多的类别来作为测试样本的类别。

在回归问题中，KNN预测一个值。使用“平均法”，将k个样本的实值输出的平均值作为测试样本的输出。一般情况下，距离度量选择欧式距离：

$$
d=\sqrt{(x_0-x_1)^2+(y_0-y_1)^2}
$$ 

三维的欧式距离公式：    

$$
d=\sqrt{(x_0-x_1)^2+(y_0-y_1)^2+(z_0-z_1)^2}
$$   


### 示例：使用k-近邻算法改进约会网站的配对系统

**项目概述**  

海伦使用约会网站寻找约会对象。经过一段时间之后，她发现曾交往过三种类型的人:

* 不喜欢的人
* 魅力一般的人
* 极具魅力的人

她希望：

1. 工作日与魅力一般的人约会
2. 周末与极具魅力的人约会
3. 不喜欢的人则直接排除掉



>收集数据：提供文本文件

海伦把这些约会对象的数据存放在文本文件 datingTestSet2.txt 中，总共有 1000 行。海伦约会的对象主要包含以下 3 种特征：

* 每年获得的飞行常客里程数
* 玩视频游戏所耗时间百分比
* 每周消费的冰淇淋公升数


文本文件数据格式如下：

    40920	8.326976	  0.953952	  3 
    14488	7.153469	  1.673904	  2
    26052	1.441871	  0.805124	  1
    75136	13.147394	 0.428964	  1
    38344	1.669788	  0.134296	  1
>准备数据：使用 Python 解析文本文件




**开发流程**     
>收集数据：提供文本文件    
准备数据：距离计算所需要的数值，最好是结构化的数据格式，使用 Python 解析文本文件    
分析数据：可以使用任何方法，使用 Matplotlib 画二维散点图     
训练算法：此步骤不适用于 k-近邻算法    
测试算法：计算错误率。   
测试样本和非测试样本的区别在于：
测试样本是已经完成分类的数据，如果预测分类与实际类别不同，则标记为一个错误。    
使用算法：产生简单的命令行程序，然后海伦可以输入一些特征数据以判断对方是否为自己喜欢的类型。

#### 1.将文本记录转换为 NumPy 的解析程序


我们在本地存储的训练集样本多为CSV格式或TXT格式，而若要在计算机中对它们进行处理，首先得将它们转换为矩阵形式。

下面的代码段就是编写了file2matrix()函数，将文本文件转换为矩阵。

In [4]:
def file2matrix(filename):
    fr = open(filename)  #读取文本内容
    
    arrayOLines=fr.readlines()   #读取文本行数
    
    numberOfLines = len(arrayOLines)           #获取文件中数据行的行数get the number of lines in the file

    returnMat = zeros((numberOfLines,3))      #创建一个numberOfLines行，3列的零矩阵         
   
    classLabelVector = []                       #创建类别列表             

    index = 0       #初始化索引值为零
    for line in arrayOLines:
       
        line = line.strip()                    #str.strip([chars])--返回移除字符串头尾指定的字符生成的新字符
                                               #s.strip()移除所有的回车字符
   
        listFromLine = line.split('\t')        #以‘\t’切割字符串，成为元素列表

        returnMat[index,:] = listFromLine[0:3]   #选取前三个元素，把他们放到特征矩阵的第index行
  
        classLabelVector.append(int(listFromLine[-1]))        #将最后一列存储到classlabelVector中；注意a[-1]表示最后一个元素
        index += 1
    return returnMat,classLabelVector   #返回数据矩阵returnMat和对应的类别classLableVector

In [6]:
datingDataMat,datingLabels=file2matrix("D:/Coding/按书籍整理程序/机器学习实战/machinelearninginaction/Ch02/datingTestSet2.txt")

datingDataMat,datingLabels[0:20]

# 《机器学习实战》书中的一个小错误，P22的datingTestSet.txt这个文件，
# 根据网上的源代码，应该选择datingTestSet2.txt这个文件。主要的区别是
# 最后的标签，作者原来使用字符串‘veryLike’作为标签，但是Python转换会
# 出现ValueError: invalid literal for int() with base 10: 'largeDoses'
# 的错误。所以改成后面的文件就可以了。后面直接用1 2 3 代表not like， 
# general like, very like。这个错误一开始用百度查不到，

(array([[4.0920000e+04, 8.3269760e+00, 9.5395200e-01],
        [1.4488000e+04, 7.1534690e+00, 1.6739040e+00],
        [2.6052000e+04, 1.4418710e+00, 8.0512400e-01],
        ...,
        [2.6575000e+04, 1.0650102e+01, 8.6662700e-01],
        [4.8111000e+04, 9.1345280e+00, 7.2804500e-01],
        [4.3757000e+04, 7.8826010e+00, 1.3324460e+00]]),
 [3, 2, 1, 1, 1, 1, 3, 3, 1, 3, 1, 1, 2, 1, 1, 1, 1, 1, 2, 3])

#### 2.分析数据：使用Matplotlib画二维散点图

首先我们使用MatplotLib制作原始数据散点图

In [3]:
import matplotlib
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(datingDataMat[:, 1], datingDataMat[:, 2], 15.0*array(datingLabels), 15.0*array(datingLabels))
plt.xlabel('玩视频游戏所耗时间的百分比')
plt.ylabel('每周消费的冰激凌公升数')
plt.show()

NameError: name 'datingDataMat' is not defined

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(datingDataMat[:, 0], datingDataMat[:, 1], 15.0*array(datingLabels), 15.0*array(datingLabels))
plt.xlabel('每年获取的飞行常客里程数')
plt.ylabel('玩视频游戏所耗时间的百分比')
plt.show()

#### 3.归一化数值

||玩视频游戏所耗时间百分比|每年获得的飞行常客里程数|每周消费的冰激凌公升数|样本分类|
|----|----|----|-----|-----|
|1|0.8|400|0.5|1|
|2|12|134000|0.9|3|
|3|0|20000|1.1|2|
|4|67|32000|0.1|2|


当度量标准不一样时，如果没有将数据归一化，可能导致数值大的特征在评价中起较大作用，而实际这几个特征的重要性是一样的。

所以为了特征重要性的评判，我们要进行数据归一化。归一化方式有很多，我们一般用：   

$$
newValue=\frac{oldValue-min}{max-min}
$$

In [ ]:
def autoNorm(dataSet):
   
    minVals = dataSet.min(0)   #找到数据最小值
    
    maxVals = dataSet.max(0)  #找到数据最大值
    
    ranges = maxVals - minVals  #最大值减最小值
    
    normDataSet = zeros(shape(dataSet)) #创建零矩阵
    
    m = dataSet.shape[0] #获取数据的行数
    normDataSet = dataSet - tile(minVals, (m,1)) #按照公式归一化
    normDataSet = normDataSet/tile(ranges, (m,1))   #element wise divide
    return normDataSet, ranges, minVals

In [ ]:
m=autoNorm(datingDataMat)
m

#### 4.分类器的测试代码


用测试数据集来测试算法的分类准确度，这里会用到刚才创建的几个函数。

程序清单：

In [ ]:
def datingClassTest():
    hoRatio = 0.10      #hold out 10%选取测试集所占样本的比例
    datingDataMat,datingLabels = file2matrix("D:\Coding\按书籍整理程序\机器学习实战\machinelearninginaction\Ch02\datingTestSet2.txt")       #load data setfrom file
    normMat, ranges, minVals = autoNorm(datingDataMat)
    m = normMat.shape[0]  #得到归一化矩阵的行数，也就是样本集中的样本总数
    numTestVecs = int(m*hoRatio) #得到测试集的个数，int()表示取整
    errorCount = 0.0
    for i in range(numTestVecs):
        classifierResult = classify0(normMat[i,:],normMat[numTestVecs:m,:],datingLabels[numTestVecs:m],3)  #前numTestVecs个数为测试集，后面为训练集，返回测试集中第i个样本分类结果
        print("the classifier came back with: %d, the real answer is: %d" % (classifierResult, datingLabels[i]))  #打印分类器分类结果和样本实际分类的类别
        if (classifierResult != datingLabels[i]): 
            errorCount += 1.0  #错误个数计数
    print("the total error rate is: %f" % (errorCount/float(numTestVecs)))
    print("the errorCount is: %d" %errorCount)

In [ ]:
rr=datingClassTest()

In [4]:
def classifyPerson():
    resultList = ['not at all', 'in small doses', 'in large doses']
    percentTats = float(input("percentage of time spent playing video games ?"))
    ffMiles = float(input("frequent filer miles earned per year?"))
    iceCream = float(input("liters of ice cream consumed per year?"))
    datingDataMat, datingLabels = file2matrix('D:\Coding\按书籍整理程序\机器学习实战\machinelearninginaction\Ch02\datingTestSet2.txt')
    normMat, ranges, minVals = autoNorm(datingDataMat)
    inArr = array([ffMiles, percentTats, iceCream])
    classifierResult = classify0((inArr-minVals)/ranges,normMat,datingLabels, 3)
    print("You will probably like this person: ", resultList[classifierResult - 1])

In [5]:
classifyPerson()

percentage of time spent playing video games ?


ValueError: could not convert string to float: 

以上构建的函数是KNN算法实现的主要框架，我们现在调用的很多算法包很多也是按照上述方式构建出来的。

当然，scikit-learn中的KNN算法肯定是优化版，速度会快很多，但核心思想却是一样的。



至此，我们完成了KNN算法原理和主要代码的学习。KNN算法是分类数据最简单最实用的方法，有广泛的应用。本篇写的比较概括，目的在于抛砖引玉，希望大家可以挖掘出KNN算法更多的东西来交流探讨。



以上构建的函数是KNN算法实现的主要框架，我们现在调用的很多算法包很多也是按照上述方式构建出来的。

当然，scikit-learn中的KNN算法肯定是优化版，速度会快很多，但核心思想却是一样的。



至此，我们完成了KNN算法原理和主要代码的学习。KNN算法是分类数据最简单最实用的方法，有广泛的应用。本篇写的比较概括，目的在于抛砖引玉，希望大家可以挖掘出KNN算法更多的东西来交流探讨。



下一节我们将进行决策树算法的学习。

--------------------------------------------------

**项目案例2: 手写数字识别系统**  


**项目概述**  


构造一个能识别数字 0 到 9 的基于 KNN 分类器的手写数字识别系统。

需要识别的数字是存储在文本文件中的具有相同的色彩和大小：宽高是 32 像素 * 32 像素的黑白图像。

**开发流程**  


* 收集数据：提供文本文件。   
* 准备数据：编写函数 img2vector(), 将图像格式转换为分类器使用的向量格式   
* 分析数据：在 Python 命令提示符中检查数据，确保它符合要求   
* 训练算法：此步骤不适用于 KNN   
* 测试算法：编写函数使用提供的部分数据集作为测试样本，测试样本与非测试样本的区别在于测试样本是已经完成分类的数据，如果预测分类与实际类别不同，则标记为一个错误    
* 使用算法：本例没有完成此步骤，若你感兴趣可以构建完整的应用程序，从图像中提取数字，并完成数字识别，美国的邮件分拣系统就是一个实际运行的类似系统      
       
         
>收集数据: 提供文本文件    



目录 trainingDigits 中包含了大约 2000 个例子，每个例子内容如下图所示，每个数字大约有 200 个样本；目录 testDigits 中包含了大约 900 个测试数据。


    

下图中采用矩阵的第一和第三列属性得到很好的展示效果，清晰地标识了三个不同的样本分类区域，具有不同爱好的人其类别区域也不同。

<img src="https://pic4.zhimg.com/80/v2-8fe726c89695482f3491c34e652e4bfd_hd.jpg",width=400,height=400>

序号玩视频游戏所耗时间百分比每年获得的飞行常客里程数每周消费的冰淇淋公升数样本分类10.84000.51212134 0000.933020 0001.1246732 0000.12

样本3和样本4的距离： $$\sqrt{(0-67)^2 + (20000-32000)^2 + (1.1-0.1)^2 }$$

归一化特征值，消除特征之间量级不同导致的影响

## KNN场景  

电影可以按照题材分类，那么如何区分 动作片 和 爱情片 呢？

1. 动作片：打斗次数更多
2. 爱情片：亲吻次数更多   

基于电影中的亲吻、打斗出现的次数，使用 k-近邻算法构造程序，就可以自动划分电影的题材类型。


<center>**每部电影的打斗镜头数、接吻镜头数以及电影评估类型**</center>

|<p align="left">电影名称|<p align="left">打斗镜头|<p align="left">接吻数目|<p align="left">电影类型|
|--|--|--|--|
|<p align="left">Californoia man|<p align="left">3|<p align="left">104|<p align="left">爱情片|
|<p align="left">He's Not Really into Dudes|<p align="left">2|<p align="left">100|<p align="left">爱情片|
|<p align="left">Beautiful Woman|<p align="left">1|<p align="left">81|<p align="left">爱情片|
|<p align="left">Kevin Longblade|<p align="left">101|<p align="left">10|<p align="left">动作片|
|<p align="left">Robo Slayer 300|<p align="left">99|<p align="left">5|<p align="left">动作片|
|<p align="left">Amped II|<p align="left">98|<p align="left">2|<p align="left">动作片|
|<p align="left">?|<p align="left">18|<p align="left">90|<p align="left">未知|


<center>**已知电影与未知电影的距离**</center>

|电影名称|与未知电影的距离|
|----|----|
|California Man|20.5|
|He's Not Really into Dudes|18.7|
|Beautiful Woman|19.2|
|Kevin Longblade|115.3|
|Robo Slayer 3000|117.4|
|Amped II|118.9|
    
    
    
　　现在根据上面我们得到的样本集中所有电影与未知电影的距离，按照距离递增排序，可以找到 k 个距离最近的电影。
假定 k=3，则三个最靠近的电影依次是， He's Not Really into Dudes 、 Beautiful Woman 和 California Man。
knn 算法按照距离最近的三部电影的类型，决定未知电影的类型，而这三部电影全是爱情片，因此我们判定未知电影是爱情片   



 

1. 假设有一个带有标签的样本数据集（训练样本集），其中包含每条数据与所属分类的对应关系。
2. 输入没有标签的新数据后，将新数据的每个特征与样本集中数据对应的特征进行比较。
    1. 计算新数据与样本数据集中每条数据的距离。
    2. 对求得的所有距离进行排序（从小到大，越小表示越相似）。
    3. 取前 k（k 一般小于等于 20）个样本数据对应的分类标签。
3. 求 k 个数据中出现次数最多的分类标签作为新数据的分类。     



其他维度依次类推

### 举个例子

假设：评价一个人的长相用[“好看”，“中等”，“难看”]来衡量。评价人的特征用：身高、体重、年龄。我们已有一个200$*$4的数据集合矩阵作为训练样本，矩阵的4列分别为：身高、体重、年龄、长相。

现得到一个测试样本，包括身高、体重、年龄三个评价特征，这时我们用kNN对这个人的长相进行分类。




**1.算法的泛化误差**

KNN算法虽然很简单，但是其泛化误差(算法推广后，机器对于未知数据的学习错误率)却是可以接受，以1NN问题(即k=1)为例,推导过程如下：

给定测试样本x,若其最近邻样本为z，1NN出错率就是x与z类别标记不同的概率，即：

$$P(error)=1-\sum_{c\in\gamma}P(c|x)P(c|z)$$

假设样本独立同分布，且对任意x和任意小正数d，在x附近d距离范围内总能找到一个训练样本z，令

$$c^*=arg\ max_{c\in\gamma}P(c|x)$$



表示贝叶斯最优分类器（以最小化总体风险为目标，对于样本的分类。通俗讲就是样本最好的分类方式，具体推导见周志华老师《机器学习》的P147页）的结果。

此时有：    


$$
\begin{aligned} % requires amsmath; align* for no eq. number
P(error) & =1-\sum_{c\in\gamma}P(c|x)p(c|z)\\
         & \approx{1-\sum_{c\in\gamma}P^2(c|x)}\\
         & \leq{1-\sum_{c\in\gamma}P^2(c^*|x)}\\
         & =(1+P(c^*|x))(1-P(c^*|x))\\
         & \leq{2}\times(1-P(c^*|x))
\end{aligned}
$$





以上得出，1NN的结构不仅简单，而且1NN的泛化错误率≤2倍的贝叶斯最优分类器错误率。


**名词解析之泛化误差**   

摘要：以前在机器学习中一直使用经验风险来逼近真实风险，但是事实上大多数情况经验风险并不能够准确逼近真实风险。后来业界就提出了泛化误差的概念（generalization error）,在机器学习中泛化误差是用来衡量一个学习机器推广未知数据的能力，即根据从样本数据中学习到的规则能够应用到新数据的能力。常用的计算方法是：用在训练集上的误差平均值-在测试集上的误差平均值。

**一：经验风险**  

机器学习本质上是一种对问题真实模型的逼近，这种近似模型也叫做一个假设。因为真实模型肯定是无法得到的，那我们的假设肯定与真实情况之间存在误差，这种误差或者误差的积累也叫做风险。

在我们选择了一个假设（或者获得一个分类器）后，为了得到真实误差的逼近，我们用分类器在样本数据上的分类结果与样本本身真实结果之间的差值来表示。这个差值叫做经验风险。

以前机器学习中经常通过经验风险的最小化作为目标,但是后来发现很多分类函数在样本集合上能够很轻易的获得100%的正确率,但是在对真实数据的分类却很糟。也表明了这种分类函数推广能力（泛化能力）差。导致这种现象的原因是：经验风险并不能够真正的逼近真实风险,因为样本集合的数目相对于真实世界要分类的数据来说就是九牛一毛。

之后统计学中就引入了泛化误差界的概念。

**二：泛化误差界**   

泛化误差界刻画了学习算法的经验风险与期望风险之间偏差和收敛速度.

真实的风险应该由两部分组成：

1：经验风险,代表分类器在给定样本上的误差（可以精确计算）。     

2：置信风险,代表我们可以在多大程度上信任分类器在未知数据上的分类结果（不可以精确计算）,因为不可以精确计算,所以只能给出一个估计区间,也因为这个泛化误差只能给出一个上界。 与置信风险相关的变量有两个：

    a)样本数量,样本数量越大表明我们的学习结果正确的可能性越大,此时置信风险越小。

    b)VC维,分类函数的VC维越大,推广能力越差,置信风险越大。

真实风险 ≤ 经验风险 + 置信风险。

现在统计学习的目标就从经验风险最小化变为经验风险与置信风险之和最小化。





-------









**算法的不足**   

KNN算法作为一种较简单的算法，它的不足之处在于：

(1)没有明显的训练过程，它是“懒惰学习”的典型代表，它在训练阶段所做的仅仅是将样本保存起来，如果训练集很大，必须使用大量的存储空间，训练时间开销为零；

(2)必须对数据集中每个数据计算距离值，实际中可能非常耗时。


**kd树**

由于上述的不足，为了提高KNN搜索的速度，可以利用特殊的数据存储形式来减少计算距离的次数。kd树就是一种以二叉树的形式存储数据的方法。

kd树就是对k维空间的一个划分。构造kd树相当于不断用垂直于坐标轴的超平面将k维空间切分，构成一系列k维超矩阵区域。kd树的每一个节点对应一个超矩阵区域。（想要深入了解的同学可以参看李航老师的《统计机器学习》P41页）


**举个例子**

给定一个二维空间的数据集：  

$$
T=\{(2,3)^T,(5,4)^T,(9,6)^T,(4,7)^T,(8,1)^T,(7,2)^T\}
$$

请构造一个平衡kd树(平衡kd树就是以中位数作为划分标准)。

解：根据点对应包含数据集T的矩阵，选择x1轴，6个数据点的x1坐标的中位数是7，以平面x1=7将空间分为左、右两个子矩阵(子节点)。

接着，左矩阵形以x2=4分为两个子矩阵，右矩阵形以x2=6分为两个子矩阵。

如此递归，最后得到如下图所示的特征空间划分。

<img src="https://pic4.zhimg.com/80/v2-289caf10e76610d4d6a1ea9a318303be_hd.jpg",width=300,height=300>

形成kd树，并利用kd树进行KNN搜索。搜索时从叶子节点往根节点搜索，具体搜索过程不展开赘述。

从文本文件中解析数据   

对未知类别属性数据集中的每个点依次执行以下操作：   
（1）计算已知类别数据集中的点与当前点之间的距离；    
（2）按照距离递增次序排序   
（3）选取与当前点距离最小的k个点    
（4）确定前k个点所在类别的出现频率    
（5）返回前k个点出现频率最高的类别作为当前点的预测分类 

In [6]:
fr=open("D:\Coding\按书籍整理程序\机器学习实战\machinelearninginaction\Ch02\datingTestSet2.txt")
arrayOLines=fr.readlines()
    #读取文本行数
numberOfLines = len(arrayOLines)            #获取文件中数据行的行数get the number of lines in the file
    #创建一个numberOfLines行，3列的零矩阵
returnMat = zeros((numberOfLines,3))        #生成对应的空矩阵prepare matrix to return
    #创建类别列表
classLabelVector = []  
arrayOLines[0:10],numberOfLines,returnMat,



NameError: name 'zeros' is not defined